### 멀티턴, 멀티쿼리, 도구가 적용된 법령 챗봇 구현
- 건축사법, 소방기본법 두 가지의 pdf 파일을 이용

In [1]:
# 필요한 라이브러리 임포트
import os
import json
from typing import List, Dict, Any, Optional, Tuple
from IPython.display import display, Markdown

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [2]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ""

### 1. pdf 파일 로드 및 벡터 DB 생성

In [4]:
# PDF 파일로부터 벡터 DB 생성 함수
def create_vectorstore_from_pdf(pdf_path: str, db_name: str) -> FAISS:
    print(f"PDF 로딩 시작: {pdf_path}")

    # PDF 로드 및 분할
    loader = PyPDFLoader(pdf_path)
    doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    docs = loader.load_and_split(doc_splitter)

    print(f"PDF 로딩 완료: {len(docs)}개 청크 생성됨")

    # 임베딩 및 벡터스토어 생성
    embedding = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(docs, embedding)

    # 벡터스토어 저장
    persist_directory = f"./DB/{db_name}"
    os.makedirs(persist_directory, exist_ok=True)
    vectorstore.save_local(persist_directory)

    print(f"{db_name} 벡터스토어 생성 완료")
    return vectorstore

In [5]:
# 이제 다운로드된 PDF 파일을 사용하여 벡터스토어 생성
building_law_db = create_vectorstore_from_pdf('건축사법(법률)(제13472호)(20160212).pdf', "building_law")
fire_law_db = create_vectorstore_from_pdf('소방기본법(법률)(제20156호)(20240731).pdf', "fire_law")

PDF 로딩 시작: 건축사법(법률)(제13472호)(20160212).pdf
PDF 로딩 완료: 119개 청크 생성됨
building_law 벡터스토어 생성 완료
PDF 로딩 시작: 소방기본법(법률)(제20156호)(20240731).pdf
PDF 로딩 완료: 142개 청크 생성됨
fire_law 벡터스토어 생성 완료


### 2. 도구 생성

In [6]:
# 도구 클래스 정의
class Tool:
    def __init__(self, name: str, description: str, vectorstore=None):
        self.name = name
        self.description = description
        self.vectorstore = vectorstore

    def __str__(self):
        return f"{self.name}: {self.description}"

In [7]:
# 도구 설정
tools = {
    "building_law": Tool(
        name="building_law",
        description=(
            "건축사법 및 관련 건축 법규에 대한 정보를 검색하고 제공합니다. "
            "건축사의 자격, 업무 범위, 설계 및 감리 규정, 건축사사무소 운영 등의 내용을 포함합니다. "
            "건축 관련 법적 절차나 규정을 확인하고 싶다면 이 도구를 사용하세요."
        ),
        vectorstore=building_law_db
    ),
    "fire_law": Tool(
        name="fire_law",
        description=(
            "소방기본법 및 화재 예방 관련 법률 정보를 검색하고 제공합니다. "
            "소방시설 기준, 화재 예방 규정, 소방 안전 관리, 소방 활동 및 긴급 구조 절차 등의 내용을 포함합니다. "
            "화재 관련 법적 요건이나 소방 규정을 확인하려면 이 도구를 사용하세요."
        ),
        vectorstore=fire_law_db
    ),
    "no_tool": Tool(
        name="no_tool",
        description=(
            "사용할 도구가 없을 경우에는 기본 LLM 답변을 작성할 것입니다. "
            "사용자의 질문을 그대로 전달하여 일반적인 정보를 제공합니다."
        )
    )
}

In [8]:
# 도구 목록 확인
for name, tool in tools.items():
    print(f"{name}: {tool.description}")

building_law: 건축사법 및 관련 건축 법규에 대한 정보를 검색하고 제공합니다. 건축사의 자격, 업무 범위, 설계 및 감리 규정, 건축사사무소 운영 등의 내용을 포함합니다. 건축 관련 법적 절차나 규정을 확인하고 싶다면 이 도구를 사용하세요.
fire_law: 소방기본법 및 화재 예방 관련 법률 정보를 검색하고 제공합니다. 소방시설 기준, 화재 예방 규정, 소방 안전 관리, 소방 활동 및 긴급 구조 절차 등의 내용을 포함합니다. 화재 관련 법적 요건이나 소방 규정을 확인하려면 이 도구를 사용하세요.
no_tool: 사용할 도구가 없을 경우에는 기본 LLM 답변을 작성할 것입니다. 사용자의 질문을 그대로 전달하여 일반적인 정보를 제공합니다.


### 3. 계획 테스트

In [9]:
# 도구 선택 계획 생성기
class ToolPlanGenerator:
    def __init__(self, tools: Dict[str, Tool], model_name="gpt-4o", temperature=0):
        self.tools = tools
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=1500)

        # 도구 설명 구성
        tool_descriptions = "\n".join([f"- {name}: {tool.description}" for name, tool in self.tools.items()])

        self.template = """
        당신은 사용자 질문을 분석하여 적절한 도구와 쿼리를 결정하는 전문가입니다.

        # 사용 가능한 도구
        {tool_descriptions}

        # 이전 대화 맥락
        이전 질문: {prev_query}
        이전 답변: {prev_response}

        # 현재 질문
        사용자 질문: {current_query}

        # 분석 지침
        1. 사용자의 질문을 분석하고, 이전 대화 맥락도 고려하세요.
        2. 질문에 여러 개의 요청이 포함되어 있다면, 각각에 대해 별도의 쿼리를 생성하세요.
        3. 각 쿼리가 어떤 도구를 사용해야 하는지 결정하세요.
        4. 질문이 어떤 도구와도 관련이 없다면 'no_tool'을 선택하세요.
        5. 마크다운 형식으로 작성하지 마세요.

        # 출력 형식
        반드시 다음 JSON 형식으로 출력하세요:
        {{
          "plan": {{
            "도구이름1": ["쿼리1", "쿼리2", ...],
            "도구이름2": ["쿼리3", "쿼리4", ...],
            ...
          }}
        }}
        """

        self.prompt = PromptTemplate(
            input_variables=["tool_descriptions", "prev_query", "prev_response", "current_query"],
            template=self.template
        )

    def generate_plan(self, prev_query: str, prev_response: str, current_query: str) -> Dict:
        # 도구 설명 구성
        tool_descriptions = "\n".join([f"- {name}: {tool.description}" for name, tool in self.tools.items()])

        try:
            # 프롬프트 준비 및 실행
            formatted_prompt = self.prompt.format(
                tool_descriptions=tool_descriptions,
                prev_query=prev_query,
                prev_response=prev_response,
                current_query=current_query
            )

            # LLM 호출
            llm_response = self.llm.invoke(formatted_prompt)
            llm_content = llm_response.content
            print(f"LLM 응답: {llm_content}")

            # JSON 파싱
            try:
                plan_json = json.loads(llm_content)
                # plan_json 형식 검증
                if "plan" not in plan_json:
                    print(f"응답에 'plan' 키가 없습니다: {plan_json}")
                    return {"plan": {"no_tool": [current_query]}}

                return plan_json
            except json.JSONDecodeError as json_err:
                print(f"JSON 파싱 오류: {str(json_err)}")
                print(f"LLM 원본 응답: {llm_content}")
                return {"plan": {"no_tool": [current_query]}}
        except Exception as e:
            print(f"계획 생성 중 오류 발생: {str(e)}")
            return {"plan": {"no_tool": [current_query]}}

In [10]:
# 계획 생성기 테스트
planner = ToolPlanGenerator(tools)
test_plan = planner.generate_plan("", "", "건축사의 자격이 궁금해")
print("생성된 계획:")
print(json.dumps(test_plan, indent=2, ensure_ascii=False))

LLM 응답: {
  "plan": {
    "building_law": ["건축사의 자격에 대한 정보"]
  }
}
생성된 계획:
{
  "plan": {
    "building_law": [
      "건축사의 자격에 대한 정보"
    ]
  }
}


### 4. 도구 선택 능력 + 멀티턴 + 멀티 쿼리 시스템

In [11]:
# 멀티 도구 RAG 시스템
class MultiToolRAG:
    def __init__(self, tools: Dict[str, Tool], model_name="gpt-4o", temperature=0.2):
        self.tools = tools
        self.tool_planner = ToolPlanGenerator(tools, model_name)
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name)

        self.prev_query = ""
        self.prev_response = ""

        # 응답 생성 프롬프트
        self.response_template = """
        당신은 건축사법 및 소방법에 대한 정보를 제공하는 AI 어시스턴트입니다.  
        사용자가 건축 관련 법률 또는 소방 관련 법률에 대한 질문을 하면, 해당 법률 문서를 기반으로 적절한 답변을 제공하세요.  

        사용자 질문: {query}

        다음은 질문에 관련된 정보입니다:
        {context}

        # 지침
        1. 제공된 정보를 바탕으로 사용자 질문에 답변하세요.
        2. 사용자 질문에 여러 질의가 있다면 각각에 대해 답변하세요.
        3. 제공된 정보가 충분하지 않으면 솔직히 모른다고 답변하세요.
        4. 답변은 한국어로 작성하세요.
        """

        self.response_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template=self.response_template
        )

    def update_conversation(self, query: str, response: str):
        self.prev_query = query
        self.prev_response = response

    def search_with_tool(self, tool_name: str, queries: List[str], num_docs=3) -> List[Document]:
        # 선택된 도구로 문서 검색
        tool = self.tools.get(tool_name)
        if not tool or tool_name == "no_tool" or not tool.vectorstore:
            return []  # no_tool이거나 벡터스토어가 없는 경우 빈 리스트 반환

        all_docs = []
        seen_contents = set()

        for query in queries:
            try:
                docs = tool.vectorstore.similarity_search(query, k=num_docs)
                print(f"{tool_name} 도구로 '{query}' 검색 완료: {len(docs)}개 문서 찾음")

                # 중복 제거
                for doc in docs:
                    if doc.page_content not in seen_contents:
                        seen_contents.add(doc.page_content)
                        # 메타데이터에 도구 및 쿼리 정보 추가
                        if not hasattr(doc, 'metadata') or doc.metadata is None:
                            doc.metadata = {}
                        doc.metadata['tool'] = tool_name
                        doc.metadata['query'] = query
                        all_docs.append(doc)
            except Exception as e:
                print(f"{tool_name} 도구로 '{query}' 검색 중 오류 발생: {str(e)}")

        return all_docs

    def query(self, current_query: str) -> Dict:
        # 도구 계획 생성
        plan = self.tool_planner.generate_plan(self.prev_query, self.prev_response, current_query)
        print(f"생성된 계획: {json.dumps(plan, indent=2, ensure_ascii=False)}")

        # 모든 도구에서 검색 수행
        all_docs = []
        for tool_name, queries in plan.get("plan", {}).items():
            tool_docs = self.search_with_tool(tool_name, queries)
            all_docs.extend(tool_docs)
            print(f"{tool_name} 도구에서 {len(tool_docs)}개 문서 검색됨")

        # 검색 결과가 없고 no_tool이 아닌 경우, no_tool 추가
        if not all_docs and "no_tool" not in plan.get("plan", {}):
            print("검색 결과가 없어 no_tool 사용")
            plan["plan"]["no_tool"] = [current_query]

        # 컨텍스트 구성
        if all_docs:
            context = "\n\n".join([f"[{doc.metadata.get('tool', 'unknown')}] 문서 {i+1}:\n{doc.page_content}"
                                 for i, doc in enumerate(all_docs)])
        else:
            context = "관련 문서가 검색되지 않았습니다."

        # 응답 생성
        formatted_prompt = self.response_prompt.format(
            query=current_query,
            context=context
        )

        result = self.llm.invoke(formatted_prompt)
        response = result.content

        # 대화 기록 업데이트
        self.update_conversation(current_query, response)

        # 결과 반환
        return {
            "query": current_query,
            "result": response,
            "plan": plan,
            "source_documents": all_docs
        }

In [12]:
# RAG 시스템 초기화
rag_system = MultiToolRAG(tools)

In [13]:
# 대화 예시 1: 건축사법 관련 질문
query1 = "건축사의 자격에 대해 알고싶어"
result1 = rag_system.query(query1)

print(f"\n질문: {query1}")
print(f"답변:\n{result1['result']}")

LLM 응답: {
  "plan": {
    "building_law": ["건축사의 자격에 대해 알고 싶어"]
  }
}
생성된 계획: {
  "plan": {
    "building_law": [
      "건축사의 자격에 대해 알고 싶어"
    ]
  }
}
building_law 도구로 '건축사의 자격에 대해 알고 싶어' 검색 완료: 3개 문서 찾음
building_law 도구에서 3개 문서 검색됨

질문: 건축사의 자격에 대해 알고싶어
답변:
건축사의 자격에 대해 알고 싶으시군요. 건축사가 되기 위해서는 다음과 같은 조건을 충족해야 합니다:

1. **학력 요건**: 
   - 대학에서 건축에 관한 소정의 과정을 이수하고 졸업한 자 및 졸업예정자 또는 「고등교육법」에 의하여 이와 동등 이상의 학력이 있다고 인정되는 자.
   - 전문대학에서 건축에 관한 소정의 과정을 이수하고 졸업한 자로서 2년 이상의 건축에 관한 실무경력을 가진 자.
   - 고등학교 또는 3년제 고등기술학교에서 건축에 관한 소정의 과정을 이수하고 졸업한 자로서 4년 이상의 건축에 관한 실무경력을 가진 자.

2. **자격시험**: 
   - 건축사가 되려는 사람은 건축사 자격시험에 합격해야 합니다. 이 시험은 건축사업무 수행에 필요한 지식과 기술을 검증하기 위해 실시됩니다.

이 외에도 실무수련을 받으려는 사람은 국토교통부장관에게 신고해야 하며, 실무수련의 과목과 절차, 평가기준 등은 대통령령으로 정해집니다.


In [14]:
# 대화 예시 2: 멀티쿼리 및 멀티턴
query2 = "음 그렇다면 그 시험에 결격사유가 있는지도 궁금하고, 소방법에는 그런 사유가 있는지도 알고싶어. 그리고 추가로 소방법의 정의도 알려줄 수 있어?"
result2 = rag_system.query(query2)

print(f"\n질문: {query2}")
print(f"답변:\n{result2['result']}")

LLM 응답: {
  "plan": {
    "building_law": ["건축사 자격시험의 결격사유가 있는지 확인"],
    "fire_law": ["소방법에 결격사유가 있는지 확인", "소방법의 정의를 알고 싶다"]
  }
}
생성된 계획: {
  "plan": {
    "building_law": [
      "건축사 자격시험의 결격사유가 있는지 확인"
    ],
    "fire_law": [
      "소방법에 결격사유가 있는지 확인",
      "소방법의 정의를 알고 싶다"
    ]
  }
}
building_law 도구로 '건축사 자격시험의 결격사유가 있는지 확인' 검색 완료: 3개 문서 찾음
building_law 도구에서 3개 문서 검색됨
fire_law 도구로 '소방법에 결격사유가 있는지 확인' 검색 완료: 3개 문서 찾음
fire_law 도구로 '소방법의 정의를 알고 싶다' 검색 완료: 3개 문서 찾음
fire_law 도구에서 6개 문서 검색됨

질문: 음 그렇다면 그 시험에 결격사유가 있는지도 궁금하고, 소방법에는 그런 사유가 있는지도 알고싶어. 그리고 추가로 소방법의 정의도 알려줄 수 있어?
답변:
사용자의 질문에 대해 다음과 같이 답변드리겠습니다.

1. **시험의 결격사유**:
   - **건축사 자격시험**: 건축사법 제9조에 따르면, 피성년후견인 또는 피한정후견인, 그리고 이 법 또는 「건축법」에 따른 죄를 범하여 금고 이상의 형을 선고받고 그 집행이 끝나거나 집행을 받지 아니한 사람은 건축사 자격을 취득할 수 없습니다.
   - **소방안전교육사 시험**: 소방법 문서에 따르면, 소방안전교육사 시험에서 부정행위를 한 사람은 해당 시험이 정지되거나 무효로 처리되며, 그 처분이 있은 날부터 2년간 시험에 응시할 수 없습니다.

2. **소방법의 정의**:
   - 소방기본법 제2조에 따르면, "소방대상물"은 건축물, 차량, 선박(「선박법」 제1조의2제1항에 따른 선박으로서 항구에 매어둔 선박만 해당) 등을 의미합니다

In [15]:
# 대화 예시 3: 도구와 관련 없는 질문
query3 = "너는 누구니?"
result3 = rag_system.query(query3)

print(f"\n질문: {query3}")
print(f"답변:\n{result3['result']}")

LLM 응답: {
  "plan": {
    "no_tool": ["너는 누구니?"]
  }
}
생성된 계획: {
  "plan": {
    "no_tool": [
      "너는 누구니?"
    ]
  }
}
no_tool 도구에서 0개 문서 검색됨

질문: 너는 누구니?
답변:
저는 건축사법 및 소방법에 대한 정보를 제공하는 AI 어시스턴트입니다. 건축 관련 법률이나 소방 관련 법률에 대한 질문이 있으시면 도와드리겠습니다. 다른 질문이 있으시면 말씀해 주세요.


### 5. 챗봇 UI

In [16]:
!pip install gradio --quiet


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import gradio as gr

# 챗봇의 응답을 처리하는 함수 (qa_chain 함수는 미리 정의되어 있어야 합니다)
def respond(message, chat_history):
    # 메시지 처리: qa_chain 또는 rag_system.query 함수 호출
    result = rag_system.query(message)
    bot_message = result['result']

    # 채팅 기록 업데이트: (사용자 메시지, 챗봇 응답) 튜플 추가
    chat_history.append((message, bot_message))
    return "", chat_history

# Gradio Blocks 인터페이스 생성
with gr.Blocks() as demo:
    # 챗봇 채팅 기록 표시 (좌측 상단 레이블 지정)
    chatbot = gr.Chatbot(label="법률 챗봇")
    # 사용자 입력 텍스트박스 (하단 레이블 지정)
    msg = gr.Textbox(label="질문해주세요!")
    # 입력창과 채팅 기록 모두 초기화할 수 있는 ClearButton
    clear = gr.ClearButton([msg, chatbot])

    # 사용자가 텍스트박스에 입력 후 제출하면 respond 함수 호출
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

# 인터페이스 실행 (debug=True로 실행하면 디버깅 정보를 확인할 수 있습니다)
demo.launch(debug=True)

C:\ProgramData\anaconda3\Lib\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


LLM 응답: {
  "plan": {
    "building_law": ["건축사의 자격에 대해 알고 싶어"]
  }
}
생성된 계획: {
  "plan": {
    "building_law": [
      "건축사의 자격에 대해 알고 싶어"
    ]
  }
}
building_law 도구로 '건축사의 자격에 대해 알고 싶어' 검색 완료: 3개 문서 찾음
building_law 도구에서 3개 문서 검색됨
LLM 응답: {
  "plan": {
    "building_law": ["건축사 자격시험의 결격사유가 있는지 확인"],
    "fire_law": ["소방법에 결격사유가 있는지 확인", "소방법의 정의에 대해 알고 싶어"]
  }
}
생성된 계획: {
  "plan": {
    "building_law": [
      "건축사 자격시험의 결격사유가 있는지 확인"
    ],
    "fire_law": [
      "소방법에 결격사유가 있는지 확인",
      "소방법의 정의에 대해 알고 싶어"
    ]
  }
}
building_law 도구로 '건축사 자격시험의 결격사유가 있는지 확인' 검색 완료: 3개 문서 찾음
building_law 도구에서 3개 문서 검색됨
fire_law 도구로 '소방법에 결격사유가 있는지 확인' 검색 완료: 3개 문서 찾음
fire_law 도구로 '소방법의 정의에 대해 알고 싶어' 검색 완료: 3개 문서 찾음
fire_law 도구에서 6개 문서 검색됨
LLM 응답: {
  "plan": {
    "no_tool": ["너는 누구니?"]
  }
}
생성된 계획: {
  "plan": {
    "no_tool": [
      "너는 누구니?"
    ]
  }
}
no_tool 도구에서 0개 문서 검색됨
Keyboard interruption in main thread... closing server.
